In [88]:
import requests
from requests import request
from requests.compat import urljoin, urlencode
from bs4 import BeautifulSoup
import re

In [89]:
url = 'https://www.scrapingcourse.com/login'
resp = request('GET', url)
resp.headers

{'Date': 'Thu, 20 Mar 2025 02:25:17 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'cloudflare', 'X-Powered-By': 'PHP/8.3.11', 'Cache-Control': 'no-cache, private', 'Cf-Cache-Status': 'DYNAMIC', 'Content-Encoding': 'gzip', 'Set-Cookie': 'AWSALB=XHQy8hAEepJ1l51tmDfVBD5atZC7+vsbSYdd2gGzR2J4a3fA5hMpfx5QwBtFNPTkO0HSzkxXA4rwVHkT9HRc11R1QCAX9LGozQkgTytFeOeI9Fh+ryfUKT+JrzrA; Path=/; Expires=Thu, 27 Mar 2025 02:25:17 GMT, AWSALBCORS=XHQy8hAEepJ1l51tmDfVBD5atZC7+vsbSYdd2gGzR2J4a3fA5hMpfx5QwBtFNPTkO0HSzkxXA4rwVHkT9HRc11R1QCAX9LGozQkgTytFeOeI9Fh+ryfUKT+JrzrA; SameSite=None; Secure; Path=/; Expires=Thu, 27 Mar 2025 02:25:17 GMT, XSRF-TOKEN=eyJpdiI6ImRHV3liL1BTclFBMmQ0N09abHFsbVE9PSIsInZhbHVlIjoiY1pORlZubjU0YUZ1SitlelN1MTZPR2JmTndhS3V1SGQycVZKUlAydWlpZ1F3N05ZaW9SWmNnYU9QUFgySDV3S3dROWM0WVQrOWpEUlRLRnJGM0lvbnJEOHdKVEg0cFBJQm5IQStESG4wR1RxMEtldnVmamtOdGtpdlU2dFFWbUQiLCJtYWMiOiJmZDJhOTAyOTU5MzBhNzAxNmQ5OTI1M2RmYjk0OTY2M2MwZWY0NDF

In [90]:
url = 'https://lms.sunde41.net'
resp = request('GET', url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [91]:
form = dom.select_one('form')
form.attrs

{'action': '/auth/login', 'method': 'POST', 'name': 'login_user_form'}

In [92]:
for el in form.select('input[name]'):
    print(el.attrs['name'])

next
email
password
remember


In [ ]:
%%writefile lms.json
{
  'id': 'example@domain.com',
  'pw': 'password'
}

Overwriting lms.json


여기서부터 코드 보고 작성하기

In [95]:
from json import load

# with open('lms.json', 'r', encoding='utf8') as fp:
#     lms = load(fp)

In [ ]:
resp = request(form.attrs['method'], urljoin(url, form.attrs['action']),
               data={'email':lms['id'], 'password':lms['pw']})

In [97]:
resp.headers['set-cookie']

'session=VoCRGCyKbOJgrnsXaGRCw3TA_5nHb1mNFvJxeBtrLKE; Expires=Sun, 20 Apr 2025 02:26:02 GMT; HttpOnly; Path=/'

In [98]:
re.search(r'<title>.+?</title>', resp.text)

<re.Match object; span=(305, 343), match='<title>대시보드 : 고려대학교 지능정보SW아카데미</title>'>

In [99]:
resp.headers['set-cookie'].split(';')[0].split('=')

['session', 'VoCRGCyKbOJgrnsXaGRCw3TA_5nHb1mNFvJxeBtrLKE']

In [100]:
c = {resp.headers['set-cookie'].split(';')[0].split('=')[0]:
     resp.headers['set-cookie'].split(';')[0].split('=')[1]}

In [101]:
resp1 = request('get', 'https://lms.sunde41.net/course/8', cookies=c)
re.search(r'<title>.+?</title>', resp1.text)

<re.Match object; span=(305, 345), match='<title>수업 게시판 : 고려대학교 지능정보SW아카데미</title>'>

In [102]:
from requests.sessions import Session

In [103]:
s = Session()

In [104]:
for k,v in c.items():
    s.cookies.set(k,v)

In [105]:
s.cookies.get_dict()

{'session': 'VoCRGCyKbOJgrnsXaGRCw3TA_5nHb1mNFvJxeBtrLKE'}

In [106]:
resp2 = s.request('get', 'https://lms.sunde41.net/course/8')
re.search(r'<title>.+?</title>', resp2.text)

<re.Match object; span=(305, 345), match='<title>수업 게시판 : 고려대학교 지능정보SW아카데미</title>'>

In [107]:
c = {resp.headers['set-cookie'].split(';')[0].split('=')[0]:
     resp.headers['set-cookie'].split(';')[0].split('=')[1]}

In [108]:
URLs = ['https://lms.sunde41.net/course/8']
Seens = []

domain = re.compile(r'lms.sunde41.net')

while URLs:
  seed = URLs.pop(0)
  Seens.append(seed)

  #resp = request('get', seed, cookies=c)
  resp = s.request('get', seed)

  if resp.status_code != 200:
    continue

  dom = BeautifulSoup(resp.text, 'html.parser')

  for a in dom.select('a[href]'):
    nurl = urljoin(seed, a['href'])
    if nurl not in URLs and nurl not in Seens and domain.search(nurl):
      URLs.append(nurl)

In [109]:
URLs

[]

In [ ]:
Seens # 게시판의 모든 자료 접근 가능

In [111]:
#Crawler
URLs = ['https://lms.sunde41.net/course/8']
Seens = []

domain = re.compile(r'lms.sunde41.net')

while URLs:
  seed = URLs.pop(0)
  Seens.append(seed)

  #resp = request('get', seed, cookies=c)
  resp = s.request('get', seed)

  if resp.status_code != 200:
    continue

  dom = BeautifulSoup(resp.text, 'html.parser')

#Scraping (수업자료 -> 앞부분 path가 동일)
  for a in dom.select('a[href]'):
    nurl = urljoin(seed, a['href'])
    if nurl not in URLs and nurl not in Seens and domain.search(nurl):
      URLs.append(nurl)

In [120]:
url = 'https://lms.sunde41.net/static/uploads/lectures/8/Database.pdf'
# '/static/uploads/lectures/8/Database.pdf'

# domain = 'test.com'
# root_path = '/home/user/www'
#                    URL       ServerLocal
# http://test.com/ => / = /home/user/www
# :www-data
#                    /database.pdf
#                              /home/user/www/database.pdf

#서버에서 보이는 URL : domain = 'test.com'
#ServerLocal에서 보이는 path : root_path = '/home/user/www'

In [121]:
resp = request('get', url) # Cookie X
resp.headers['content-type']

'application/pdf'

In [122]:
re.search(r'application/(pdf)', resp.headers['content-type']).group(1)

'pdf'

In [123]:
with open(re.search(r'.+/(.+)$', url).group(1), 'wb') as fp:
    fp.write(resp.content)

In [124]:
!ls -al *.pdf

-rw-r--r--  1 gokyulueau  staff  4660853 Mar 20 11:37 Database.pdf


In [74]:
resp2 = s.request('get', 'https://lms.sunde41.net/qna/new/8')
re.search(r'<title>.+?</title>', resp2.text)

<re.Match object; span=(305, 341), match='<title>질문 : 고려대학교 지능정보SW아카데미</title>'>

In [75]:
dom = BeautifulSoup(resp2.text, 'html.parser')

In [76]:
form = dom.select_one('form')
form.attrs['action'], form.attrs['method']
# RESTful API

('/qna/new/8', 'post')

In [77]:
for el in form.select('[name]'):
    print(el.name, el.attrs['name'])

input title
textarea description


In [78]:
nurl = urljoin(resp2.url, form.attrs['action'])
# LMS 도움게시판에 글 쓰는 요청이니 쓰지 말 것
# s.request(form.attrs['method'], nurl, data={'title':'수업 테스트 중', 'description':'신경쓰지 마세요'})

In [79]:
url = 'https://lms.sunde41.net/qna/detail/80' # db에 들어가는 글 번호
resp = s.request('get', url)
dom = BeautifulSoup(resp.text, 'html.parser')
form = dom.select_one('form')
form.attrs['action'], form.attrs['method']

('/qna/detail/80/comments/edit/83', 'post')

In [80]:
form.select_one('[name]').attrs['name']

'description'

In [ ]:
# 해당 글에 댓글 달도록 요청
# s.request(form.attrs['method'], url, data={'description':'3월 20일 테스트'})

In [82]:
url = 'https://lms.sunde41.net/qna/detail/80' # db에 들어가는 글 번호
resp = s.request('get', url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [83]:
dom.select('.page-body .card:nth-child(1) [data-description]')

[<div class="mt-2 zoom" data-description="83"> 키키 </div>,
 <div class="mt-2 zoom" data-description="85"> 테스트 </div>,
 <div class="mt-2 zoom" data-description="86"> 테스트 </div>,
 <div class="mt-2 zoom" data-description="88"> 3월 20일 테스트 </div>,
 <div class="mt-2 zoom" data-description="89"> test </div>,
 <div class="mt-2 zoom" data-description="90"> 테스트 </div>,
 <div class="mt-2 zoom" data-description="92"> 테스트 </div>,
 <div class="mt-2 zoom" data-description="93"><p>ㅁㄴㅇㄹ</p></div>]

In [84]:
dom.select('.page-body .card:nth-child(1) a[href]')

# 확대/축소
# callEdit
# /qna/detail/80/comments/delete/88
#             --(질문id)          --(답변id)

[<a class="btn-action" data-bs-html="true" data-bs-placement="left" data-bs-toggle="tooltip" href="#" id="maximize1" title="최대/최소화"><svg class="icon icon-tabler icon-tabler-arrows-maximize" fill="none" height="16" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24 24" width="16" xmlns="http://www.w3.org/2000/svg"><path d="M0 0h24v24H0z" fill="none" stroke="none"></path><path d="M16 4l4 0l0 4"></path><path d="M14 10l6 -6"></path><path d="M8 20l-4 0l0 -4"></path><path d="M4 20l6 -6"></path><path d="M16 20l4 0l0 -4"></path><path d="M14 14l6 6"></path><path d="M8 4l-4 0l0 4"></path><path d="M4 4l6 6"></path></svg></a>,
 <a class="btn btn-warning btn-sm btn-icon" href="#" onclick="callEdit(88)"><svg class="icon icon-tabler icon-tabler-edit" fill="none" height="16" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24 24" width="16" xmlns="http://www.w3.org/2000/svg"><path d="M0 0h24v24H0z" fil

In [85]:
dom.select_one('.page-body .card:nth-child(1) .card-actions')

<div class="card-actions btn-actions"><a class="btn-action" data-bs-html="true" data-bs-placement="left" data-bs-toggle="tooltip" href="#" id="maximize1" title="최대/최소화"><svg class="icon icon-tabler icon-tabler-arrows-maximize" fill="none" height="16" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24 24" width="16" xmlns="http://www.w3.org/2000/svg"><path d="M0 0h24v24H0z" fill="none" stroke="none"></path><path d="M16 4l4 0l0 4"></path><path d="M14 10l6 -6"></path><path d="M8 20l-4 0l0 -4"></path><path d="M4 20l6 -6"></path><path d="M16 20l4 0l0 -4"></path><path d="M14 14l6 6"></path><path d="M8 4l-4 0l0 4"></path><path d="M4 4l6 6"></path></svg></a></div>

In [ ]:
# 해당 댓글 지우기
# s.request('get', urljoin(resp.url, '/qna/detail/80/comments/delete/88'))

<Response [200]>

In [87]:
s.cookies.clear()